In [ ]:
##############################################
#                INDEXING                    #
##############################################

In [1]:
# Loading
from langchain_community.document_loaders import PDFMinerLoader
import re

def metin_önişleme(documents):
    for i in range(len(documents)):
        documents[i].page_content = re.sub(r'\.{2,}', '.', documents[i].page_content)
        documents[i].page_content = documents[i].page_content.replace("\n", " ")
        documents[i].page_content = re.sub(r'\s{3,}', " ", documents[i].page_content)
    return documents

loader = PDFMinerLoader("data/YapayZeka.pdf")
data = loader.load()

document = metin_önişleme(data)
print(document)

[Document(metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2006-11-05T22:13:15+02:00', 'moddate': '2006-11-05T22:13:15+02:00', 'title': 'Microsoft Word - yapay_zeka_icerik1_1.6.doc', 'author': 'sonmez', 'total_pages': 14, 'source': 'data/yapay_zeka_icerik1_1.6.pdf'}, page_content='1.GİRİŞ  Yapay  zekayı  anlamak  bilgisayarda  klasik  şekilde  veri  işlemek  düşüncesinden uzaklaşmayı gerektirir.  Burada söz konusu olan bilgisayarların programları ile klasik algoritmik  işlemleri  yapmasından  öte  daha  can  alıcı  özelliklerle ortaya  çıkmasıdır. Şöyle ki, bir bilgisayarınız var ama klavyesi yok, dolayısıyla konuştuğunuzu anlıyor, ona  göre  ilgili  komutları  yerine  getiriyor  ve  cevap  veriyor. Tabiî  ki  sonucu  size kullanıcı  uyumlu,  sesli  ve/veya  zengin  içerikli  grafik  ekran  gibi  çıkış  birimlerinden sunuyor. Daha  ileri  giderek,  bilgisayardan  tanımladığınız  işlevleri  yerine  getirecek program  üretme

In [19]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024,
    chunk_overlap=200)

# Make splits
splits = text_splitter.split_documents(document)
print(f"Chunk Sayısı: {len(splits)}")
print("---"*30)
for chunk in splits:
    print(chunk.page_content)
    print("---"*30)

Chunk Sayısı: 16
------------------------------------------------------------------------------------------
1.GİRİŞ  Yapay  zekayı  anlamak  bilgisayarda  klasik  şekilde  veri  işlemek  düşüncesinden uzaklaşmayı gerektirir.  Burada söz konusu olan bilgisayarların programları ile klasik algoritmik  işlemleri  yapmasından  öte  daha  can  alıcı  özelliklerle ortaya  çıkmasıdır. Şöyle ki, bir bilgisayarınız var ama klavyesi yok, dolayısıyla konuştuğunuzu anlıyor, ona  göre  ilgili  komutları  yerine  getiriyor  ve  cevap  veriyor. Tabiî  ki  sonucu  size kullanıcı  uyumlu,  sesli  ve/veya  zengin  içerikli  grafik  ekran  gibi  çıkış  birimlerinden sunuyor. Daha  ileri  giderek,  bilgisayardan  tanımladığınız  işlevleri  yerine  getirecek program  üretmesini  sağlayabiliyorsunuz.  Evet,  bunlar  yakın  zamana  kadar  her  biri birer  hayal  olmaktan  öteye  gitmeyen  düşüncelerdi. Fakat  bunların  tümü  olmasa  da kısmen  gerçekleştiğini  görmekteyiz.  Bu  alandaki  gelişmenin  bilgisaya

In [3]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

load_dotenv()


vectorstore = FAISS.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)
vectorstore.save_local("YapayZeka")

In [ ]:
##############################################
#                 RETRIEVAL                  #
##############################################

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
vb = FAISS.load_local(
    folder_path="YapayZeka",
    embeddings=OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)

In [7]:
# Retriever tanımlanması
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [8]:
# Retriever ile ilgili belgelerin getirilmesi
docs = retriever.get_relevant_documents("Yapay Zeka'nın amacı nedir ?")
len(docs)
print(docs)

C:\Users\melih\AppData\Local\Temp\ipykernel_5476\3673674139.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Yapay Zeka'nın amacı nedir ?")


[Document(id='e66d6bfa-f748-46f5-8a44-757cb9088ee8', metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2006-11-05T22:13:15+02:00', 'moddate': '2006-11-05T22:13:15+02:00', 'title': 'Microsoft Word - yapay_zeka_icerik1_1.6.doc', 'author': 'sonmez', 'total_pages': 14, 'source': 'data/yapay_zeka_icerik1_1.6.pdf'}, page_content='yapay zekayı ele almıştır yada yapay zeka için özel ekler yayınlamışlardır. Ortaya düzinelerce yapay zeka kitabı çıkmıştır. Bir çok yapay zeka raporu düzenli olarak yayınlanmış ve dünya çapında bu konuda konferanslar düzenlenmiştir. 1.2. YAPAY ZEKA TANIMI \x0cSayısız uygulama alanı olan “Yapay Zeka” kavramının doğal olarak pek çok tanımı da olacaktır.  Aşağıda bunlardan bir bölümü verilmiştir: YAPAY  ZEKA\'nın  amacı,  normal  olarak  insan  zekasını  gerektiren  görevleri yapabilecek makinalar yapmaktır. YAPAY  ZEKA  araştırmalarının  amacı,  insan  varlığında  gözlemlediğimiz  ve  "akıllı davranış" ola

In [ ]:
##############################################
#                 GENERATION                 #
##############################################

In [9]:
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
# Prompt
template = """
    1- Aşağıdaki bağlamları kullanarak soruyu yanıtlayın.
    2- Sağlanan bağlamdan en ayrıntılı şekilde soruyu yanıtlayın.
    3- En fazla 5 cümle kullanın.
    4- Cevap sağlanan bağlamda yoksa, "Üzgünüm cevabı bulamadım..." demelisiniz.
    5- Yanıt Türkçe olmalıdır.
    Context:\n {context}\n
    Question: \n{question}\n
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    1- Aşağıdaki bağlamları kullanarak soruyu yanıtlayın.\n    2- Sağlanan bağlamdan en ayrıntılı şekilde soruyu yanıtlayın.\n    3- En fazla 5 cümle kullanın.\n    4- Cevap sağlanan bağlamda yoksa, "Üzgünüm cevabı bulamadım..." demelisiniz.\n    5- Yanıt Türkçe olmalıdır.\n    Context:\n {context}\n\n    Question: \n{question}\n\n'), additional_kwargs={})])

In [11]:
# LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [12]:
# Chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
# Run
rag_response = rag_chain.invoke("Yapay Zeka'nın amacı nedir ?")
print(f"RAG Response: \n{rag_response}")

Elcevab : 
Yapay Zeka'nın amacı, üç ana başlık altında toplanabilir: makinaları daha akıllı hale getirmek, zekanın ne olduğunu anlamak ve makinaları daha faydalı hale getirmektir. Yapay Zeka, insan tarafından yapıldığında zeka olarak adlandırılan davranışların makinalar tarafından da yapılmasını hedefler. Ayrıca, insan aklının nasıl çalıştığını gösteren bir kuram olarak da değerlendirilebilir. Yapay Zeka'nın en büyük amacı, insan zekasını taklit eden makineler yapmaktır. Ancak mevcut ticari yapay zeka ürünleri, bu hedefe ulaşmada henüz tam anlamıyla başarılı olamamıştır.
